**💡 To better engage gray mass we suggest you turn off Colab AI autocompletion in `Tools > Settings > AI Assistance`**

In [1]:
%%capture
try:
    import pytensor_workshop
except ModuleNotFoundError:
    !pip install git+https://github.com/pymc-devs/pytensor-workshop.git

In [2]:
import numpy as np
from scipy.optimize import minimize, root
from sklearn.datasets import make_regression

import pytensor
import pytensor.tensor as pt

from pytensor.graph.fg import FunctionGraph
from pytensor.graph.basic import explicit_graph_inputs
from pytensor.tensor.math import sqr as pt_sqr, Sum, CAReduce
from pytensor.tensor.elemwise import Elemwise
from pytensor.scalar.basic import Sqr

In [3]:
from pytensor_workshop import test

# Optimization with Pytensor

A common task in numerical computation is optimization of an objective function. In the Python ecosystem, we'll typically use `scipy.optimize.minimize` to accomplish this. 

If you're not familiar with it, `minimize` has a ton of options, not all of which are particularly well documented.  At its simplest, though, it takes at least 3 arguments:

1. `fun` - A function with inputs `x, *args`, which outputs a single scalar value. `x` is a vector of parameters to be chosen. `*args` are any other arguments that `fun` requires, but they are *not* optimized. `x` will be chosen so that `fun(x, *args)` becomes as small as possible. 

2. `x0` - The initial values of the `x` values. This should be a sequence (if `x` is vector valued), otherwise a scalar.

3. `args` - A tuple of anything you like, to be passed into `fun` via tuple-unpacking. 

3. `method` - A string selecting an optimization algorithm to use. Each method has different requirements, and we'll talk about a couple as we go.

## Warmup: minimization with numpy

Just so we're on the same page, let's start by just optimizing $f(x) = (x - a)^2$. Obviously this takes a minimum at $x = a$. In numpy, we'd actually write a function, then pass this into `minimize`.

We choose `method="nelder-mead"`, which is a [gradient-free optimization method](https://en.wikipedia.org/wiki/Nelder-Mead). That's nice because in numpy we don't have easy access to gradients!

In [4]:
def objective(x, a):
    return (x - a) ** 2

res = minimize(fun=objective, x0=[0.0], args=(3,), method='nelder-mead')
res

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 9.663546088957395e-30
             x: [ 3.000e+00]
           nit: 28
          nfev: 56
 final_simplex: (array([[ 3.000e+00],
                       [ 3.000e+00]]), array([ 9.664e-30,  3.906e-09]))

## Exercise 1: Use a pytensor function

For the first exercise, we'll minimize $f(x) = (x - a)^2$ again, but this time using a pytensor function. 

In [5]:
x = pt.dvector('x')  # This has to be a vector because of how scipy works, even if it's shape (1,)
a = ...              # This can be a scalar
f_x = ...

# f = pytensor.function([x, a], f_x)


@test
def test_simple_minimize(f, a=3):
    res = minimize(fun=f, x0=0, args=(a,), method='nelder-mead')
    
    np.testing.assert_allclose(res.fun, 0.0, atol=1e-8, rtol=1e-8)
    np.testing.assert_allclose(f(res.x, a), 0.0, atol=1e-8, rtol=1e-8)
    
# test_simple_minimize(f, 3)

## Exercise 2: A more complex function

The original function we looked at was pretty boring. Let's look at minimization of sum of squares, which is near and dear to all of our hearts. 

To make it even more interesting, I demand you write the whole thing in linear algebra -- no `sum` or `square` allowed anywhere!

In [6]:
X = pt.dmatrix('X')
y = pt.dvector('y')
beta = pt.dvector('beta')

residuals = ...
sse = ...

f_sse = ...

@test
def test_minimize_sum_of_squares(f_sse):        
    fg = f_sse.maker.fgraph
    if any(isinstance(node.op, CAReduce) or 
           (isinstance(node.op, Elemwise) and isinstance(node.op.scalar_op, Sqr)) 
           for node in fg.toposort()):
        raise ValueError("No Cheating! Use linear algebra!")
    
    X, y, true_beta = make_regression(n_samples=100, 
                                      n_features=5,
                                      n_informative=5, 
                                      bias=10,
                                      noise=0,  # noise zero is important so we get exactly the right answer
                                      coef=True)
    X = np.c_[np.ones(100), X]
    true_beta = np.r_[10., true_beta]
    
    res = minimize(fun=f_sse, 
                   x0=np.random.normal(0, 10, size=(6,)),
                   args=(X, y), 
                   method='powell')

    np.testing.assert_allclose(res.x, true_beta)

# test_minimize_sum_of_squares(f_sse)

## Exercise 3: One more with direction

As problems get larger, using gradient free methods is not going to cut it. Luckily, pytensor can handle the gradient computations for us using `pt.grad`. We only require a scalar loss function.

From scipy's perspective, we add a new argument to `minimize`: `jac`. It drives me nuts, because it's not actually a jacobian, it's a gradient. Maybe someone mathy can justify the name to me. Anyway, `jac` is either a `bool` or `Callable`, and the behavior changes depending on which you pass:

- If `jac` is `False`, we don't use gradients.
- If `jac` is Callable, it should have the **same signature** as `fun`, and return $\frac{\partial f(x)}{\partial x}$
- If `jac` is `True`, `fun` is assumed to be a *fused* function that returns two values: the loss, and the gradients.

Let's start by making separate functions. Recycling the variables you created above, make a new funciton `f_sse_grad`.

Once we have gradients, we can use Newton-Raphson style minimization algorithms. `BFGS` is a popular one that works well on many problems.

In [7]:
sse_grad   = ...
f_sse_grad = ...

@test
def test_minimize_sum_of_squares_with_grad(f_sse, f_sse_grad):        
    fg = f_sse.maker.fgraph
    if any(isinstance(node.op, CAReduce) or 
           (isinstance(node.op, Elemwise) and isinstance(node.op.scalar_op, Sqr)) 
           for node in fg.toposort()):
        raise ValueError("No Cheating! Use linear algebra!")
    
    X, y, true_beta = make_regression(n_samples=1000, 
                                      n_features=100,
                                      n_informative=100, 
                                      bias=10,
                                      noise=0,  # noise zero is important so we get exactly the right answer
                                      coef=True)
    X = np.c_[np.ones(1000), X]
    true_beta = np.r_[10., true_beta]
    
    res = minimize(fun=f_sse, 
                   x0=np.random.normal(0, 10, size=(101,)),
                   args=(X, y),
                   jac=f_sse_grad,
                   method='BFGS')

    np.testing.assert_allclose(res.x, true_beta)
    np.testing.assert_allclose(f_sse_grad(res.x, X, y), 0.0, atol=1e-4, rtol=1e-4)
    
# test_minimize_sum_of_squares_with_grad(f_sse, f_sse_grad)

Adding gradients is nice because it allows convergence in fewer iterations. Let's make one last function, this time a fused objective. It should take `beta, X, y` and return `sse, sse_grad`

In [8]:
f_fused = ...

In [9]:
@test
def test_minimize_sum_of_squares_fused(f_fused):        
    X, y, true_beta = make_regression(n_samples=1000, 
                                      n_features=100,
                                      n_informative=100, 
                                      bias=10,
                                      noise=0,  # noise zero is important so we get exactly the right answer
                                      coef=True)
    X = np.c_[np.ones(1000), X]
    true_beta = np.r_[10., true_beta]
    
    res = minimize(fun=f_fused, 
                   x0=np.random.normal(0, 10, size=(101,)),
                   args=(X, y),
                   jac=True,
                   method='BFGS')
    
    np.testing.assert_allclose(res.x, true_beta)
    np.testing.assert_allclose(f_fused(res.x, X, y)[1], 0.0, atol=1e-4, rtol=1e-4)
    
    # Show that providing gradients leads to faster convergence
    res2 = minimize(fun=lambda x, *args: f_fused(x, *args)[0], 
               x0=np.random.normal(0, 10, size=(101,)),
               args=(X, y),
               jac=False,
               method='BFGS')

    print(f'Number of function evaluations WITH gradients: {res.nfev}')
    print(f'Number of function evaluations WITHOUT gradients: {res2.nfev}')

# test_minimize_sum_of_squares_fused(f_fused)

## Exercise 4: What do German mercenaries have to do with anything?

The last bit of information we can include is the Hessian matrix. There are two ways to do this:

1. We can make a function that directly computes the entire hessian. This straight-forward, but really only feisible on small-scale problems. It's shape $k \times k$, where $k$ is the number of parameters being optimized. For big $k$, things get gnarly fast!

2. Provide a "hessp". This is a vector which represents the product between the hessian and some arbitrary direction vector. This takes a bit more work to set up, but results in much more effecient computation, because we never need to make the whole $k \times k$ hessian. 

Let's use the hessian function directly first, and let's also up the complexity of things. Instead of minimizing the sum of squares, let's instead find the MAP of a Bayesian model built using PyMC. For the model, let's make a simple Bernoulli GLM:

$$
\begin{align}
y &\sim \text{Bernoulli}(p) \\
p &= \text{logit}^{-1}(X \beta) \\
\beta & \sim N(0, 1) \\
\end{align}
$$

In [10]:
import pymc as pm

y = pt.dvector('y')
X = pt.dmatrix('X')

with pm.Model() as m:
    
    beta = ... # Make this size (10,)
    p    = ...
    obs  = ...

In [11]:
## Uncomment these once you implement the model!
# beta_val = m.rvs_to_values[beta]

# inputs = [beta_val, X, y]
# neg_logp = -m.logp()

grad = ...
hess = ... #Hint: you can use pytensor.gradient.hessian directly on the objective, 
           # or be clever and use pytensor.gradient.jacobian, but not on the objective :O

f_fused = ...
f_hess  = ...

In [12]:
@test
def test_MAP_estimation(f_fused, f_hess):
    rng = np.random.default_rng(123)
    X = rng.normal(size=(10000, 9))
    X = np.c_[np.ones(10000), X]

    true_beta = rng.normal(scale=0.1, size=(10,))
    true_p = pm.math.invlogit(X @ true_beta).eval()
    y = rng.binomial(1, true_p)
    
    res = minimize(fun=f_fused,
               x0=np.zeros_like(true_beta),
               args=(X, y),
               jac=True,
               hess=f_hess,
               method='trust-ncg',
               )
        
    np.testing.assert_allclose(res.x, true_beta, atol=0.1, rtol=0.1)
    np.testing.assert_allclose(f_fused(res.x, X, y)[1], 0.0, atol=1e-4, rtol=1e-4)
    

# test_MAP_estimation(f_fused, f_hess)

# Constrained Optimization

Another important type of problem is constrained optimization. Here, we want to optimize subject to a set of equality or inequality constraints. `scipy.minimize` offers several solvers that allow one to do this, the most robust of which is `trust-constr`. Pytensor can again help us several ways:

1. We can write functions symbolically, making it easier to organize parameters and constraints
2. We can get derivatives of the constraints, which are used by `trust-constr`.

Let's start with a new problem. The marketing guys like to work with a function called the Michaelis-Mente curve. It has the following form:

$$ 
f(x, \alpha, \lambda) = \frac{\alpha x}{x + \lambda}
$$

They jointly optimize stacks of these curves by choosing $x$ (the amount of advertising spend), subject to known $\alpha$ and $\lambda$, plus budget constraints. Let's slowly build up to this task.

## Exercise 5: Quick Marketing Warmup

Use pytensor to implement the Michaelis-Mente curve. $x$, $\alpha$, and $\lambda$ should all be scalars for now.

In [14]:
x, alpha, lam = ..., ..., ...  # hint, pt.dscalars (with an s!) is nice for making many variables quickly
mm_curve = ...
f_mm = ...

@test
def test_scalar_michaelis_mente(f_mm):
    test_cases = [(0., 0., 1.), (1., 1., 1.), (0.5, 0.5, 0.5)]
    expected_outputs = [0.0, 0.5, 0.25]
    for (x, α, λ), output in zip(test_cases, expected_outputs):
        np.testing.assert_allclose(f_mm(x, α, λ), output)
        

# test_scalar_michaelis_mente(f_mm)

## Exercise 6: Vectorization!

A scalar function isn't very useful -- we can't even draw curves without a loop. Use `vectorize_graph` to replace `x` with `x_vector`

In [15]:
from pytensor.graph.replace import vectorize_graph
x_vector  = ...
vector_mm = ...

f_mm_vec = ...

@test
def test_vector_michaelis_mente(f_mm):
    x_grid = np.linspace(0, 1, 100)
    alpha_val = 0.5
    lambda_val = 0.3
    expected_outputs = (x_grid * alpha_val) / (x_grid + lambda_val)
    np.testing.assert_allclose(f_mm(x_grid, alpha_val, lambda_val), expected_outputs)        

# test_vector_michaelis_mente(f_mm_vec)

## Exercise 6 1/2: Plot the function

While not necessary, it's always good to actually look a the functions we're working with. The next cell uses an ipython widgets to let you play with the function. Run it and play around with the function, getting a sense for the role of alpha and lambda:

In [16]:
PLAY_WITH_FUNCTION = False

if PLAY_WITH_FUNCTION:
    %matplotlib notebook
    import matplotlib.pyplot as plt
    import ipywidgets as widgets

    x_grid = np.linspace(0, 1, 100)
    fig, ax = plt.subplots(figsize=(8, 6), dpi=77)
    line, = ax.plot(x_grid, f_mm_vec(x_grid, 0.5, 0.1))

    alpha_slider  = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01, description='α:')
    lambda_slider = widgets.FloatSlider(value=0.5, min=1e-4, max=1.0, step=0.01, description='λ:')

    @widgets.interact(α=alpha_slider, 
                      λ=lambda_slider)
    def update(α=0.5, λ=0.1):
        new_curve = f_mm_vec(x_grid, α, λ)
        line.set_ydata(new_curve)
        fig.canvas.draw_idle()

    update()

## Exercise 7: More vectorization!

In marketing applications, we will have one of these curves per product and channel. That means we need to vectorize not only over `x` (because we want a function that generates curves, not merely point), but also over $\alpha$ and $\lambda$.

This exercise will highlight one of the sharp edges of Pytensor -- you are *not* allowed to do so-called "runtime broadcasting"! Our function is now going to have signature `(n),(c),(c)->(n,c)`. That is, `x` will have shape `(n,)` (the number of points to evaluate on, while $\alpha$ and $\lambda$ will have shape `(c,)` (the number of marketing channels we're modeling). So we have to make sure the pytensor function as written actually does this.

Suppose we were doing this in numpy. To make the broadcasting work, you would add a dummy dimension to `x`, as in:

```py
vec_mm = x[:, None] * alpha / (x[:, None] + lam)
```

If you naively use `vectorize_graph` to switch $\alpha$ and $\lambda$ to vectors, you will not be able to get this dummy dimension in your graph. 

Instead, you will need to add `x` *with the extra dimension* when you call `vectorize_graph`.

In [18]:
alpha_vector, lam_vector = ..., ...

vector_mm_2 = ...
f_mm_vec_2 = ...

@test
def test_fully_vectorized_michaelis_mente(f_mm):
    x_grid = np.linspace(0, 1, 100)
    alphas = np.array([0.5, 0.3, 0.3, 0.5, 0.5, 0.4, 0.4, 0.3])
    lambdas = np.array([0.3, 0.6, 0.3, 0.1, 0.2, 0.9, 0.3, 0.7])
    expected_outputs = (x_grid[:, None] * alphas) / (x_grid[:, None] + lambdas)
    
    your_output = f_mm(x_grid, alphas, lambdas)
    
    assert your_output.shape == (100, 8)
    np.testing.assert_allclose(your_output, expected_outputs)        

# test_fully_vectorized_michaelis_mente(f_mm_vec_2)

## Exercise 7.5: Can we optimize something?

No, not yet. First, we need to get a feeling for the contours of the problem.

The Michaelis Mente function is something like the "utility" of spending money on each channel. Suppose we have 8 channels with known parameters $\alpha$ and $\lambda$. The game becomes the following: choose allocations $x_1, x_2, \dots, x_8$ across these 8 channels such that the sum of their utilities is maximized.

Since the Michaelis Mente is monotonically increasing, the wise-assed among you will note that utility is maximized by allocating infinity dollars to all of the channels. So for this to actually be interesting, we need to have a budget constraint. Specifically, *the sum of all allocations should be less than or equal to 1*.

Coming up with good allocations is not so easy. Play with the widget in the next cell to see how well you can do.

In [20]:
PLAY_WITH_FUNCTION = False

if PLAY_WITH_FUNCTION:
    %matplotlib notebook
    import matplotlib.pyplot as plt
    import ipywidgets as widgets

    alphas = np.array([0.5, 0.3, 0.3, 0.5, 0.5, 0.4, 0.4, 0.3])
    lambdas = np.array([0.3, 0.6, 0.3, 0.1, 0.2, 0.9, 0.3, 0.7])

    x_grid = np.linspace(0, 1, 100)
    
    allocations = np.zeros_like(alphas)
    initial_utility = np.diagonal(f_mm_vec_2(allocations, alphas, lambdas))
    
    HIGH_SCORE = initial_utility.sum()
    
    fig, ax = plt.subplots(1, 2, figsize=(12, 6), dpi=77)
    ax[0].plot(x_grid, f_mm_vec_2(x_grid, alphas, lambdas), label=[f'x{i+1}' for i in range(8)])
    ax[0].set_title('Utility from Channel Allocations')
    ax[0].legend()
    scatter = ax[0].scatter(allocations, initial_utility)
    
    spend_bar, utility_bar = ax[1].bar(['Total Spend', 'Total Utility'], 
              [allocations.sum(),                          
               np.trace(f_mm_vec_2(allocations, alphas, lambdas))])
    
    ax[1].axhline(1.0, ls='--', c='k', lw=2, label='Total Budget')
    high_score_line = ax[1].axhline(HIGH_SCORE, c='tab:red', lw=2, label='High Score')
    ax[1].set(ylim=(0, 1.5), title=f'High Score: {HIGH_SCORE}')
    ax[1].legend()
    
    sliders  = {f'x_{i}': widgets.FloatSlider(value=allocations[i], min=0.0, max=1.0, step=0.001, 
                                    description=f'x_{i+1} spend:')
               for i in range(8)}
    
    @widgets.interact(**sliders)
    def update(**allocations):
        global HIGH_SCORE
        
        allocations = np.array(list(allocations.values()))
        if len(allocations) > 0:
            total_spend = allocations.sum()
            out = f_mm_vec_2(allocations, alphas, lambdas)
            y_vals = np.diagonal(out)
            scatter.set_offsets(np.c_[allocations, y_vals])
            
            spend_bar.set_height(total_spend)
            
            if total_spend > 1.0:
                spend_bar.set_color('tab:red')
            else:
                spend_bar.set_color('tab:blue')
            
            utility = np.trace(out)
            utility_bar.set_height(utility)
            if utility > HIGH_SCORE and total_spend < 1.0:
                HIGH_SCORE = utility
                ax[1].set_title(f'High Score: {HIGH_SCORE}')
                high_score_line.set_ydata([utility, utility])
            
            fig.canvas.draw_idle()

    update()

## Exercise 8: Constrainted Optimization

Finally, let's optimize. Some considerations.

### Scipy Constraints

Scipy allows you to define constrains via the `constraints` argument of `minimize`. There are many ways to do it, see here for the [docs](https://docs.scipy.org/doc/scipy/tutorial/optimize.html#constrained-minimization). Here, I present what I consider to be the "easiest" (though this is admittedly relative).

To define constraints, we pass a list of dictionaries. Each dictionary is a constraint, and must have the following keys:

- `"type"`. This is either `"eq"`, for an Equality constraints, for `"ineq`", for an inequality constraint. Importantly, inequality constraints are **always** $f(x) \geq 0$. So if you wanted $x \leq 1$, you pass $1 - x$. If you want $x > 0$, you just pass $x$, and so on.
- `"fun"`. A callable that computes the constraint
- `"jac"`. A callable that returns the jacobain of the constraint with respect to the control variable. This is technically optional, but since we have pytensor, we're always going to give it.
- `"args"`. Additional arguments that are passed to `fun`. Note that these are **not** the same as the `args` you define for the objective function! If you want those, you have to pass them again.

### Unused arguments to `pytensor.function`

Until now, we've been picky about what we pass as input to `pytensor.function`. Here, I want to encourage you to use loops to compile a bunch of functions in one go. This is a solution that scales to lots of constraints, which is a case we're interested in working towards.

As a result, we want to pass the set union of all `args` across all constraints to all constraints. It might be that not all constraints need all args! For example, imagine we have two constraints:

1. Total allocation equals total budget
2. All allocations are non-negative

In this case, we need to pass the `budget` variable to constraint 1, but not to constraint 2.

Another option is to pass it to both, but set `on_unused_input="ignore"`. When you do this, pytensor will allow you to pass any number of symbolic inputs to `pytensor.function`, *even if they are not used by the outputs*!

### Jacobian matrices

So far we've been using `pt.grad` to get gradients, because we're always working with a scalar loss function. Constraints can be systems of equations, so if we just call `pt.grad(constraint, x_vector)`, we'll (potentially) get the wrong answer.

To compute the jacobian matrix, use `pytensor.gradient.jacobian`. This function is smart, in that if you pass a scalar function as the cost, you will get back the same thing as if you had called `pt.grad`. So it's always "safe" to use it.

In [22]:
objective = ... # -pt.trace(vector_mm_2)
budget = ...  # This is a symbolic value


# Make two constraints: 
# 1. Total allocation equals total budget
# 2. All allocations are non-negative (hint: you have to include the 0 to make pytensor happy)
constraint_types = ['eq', 'ineq']
constraint_graphs = [..., 
                     ...]


objective_grad = ...
objective_hess = ...

constraint_jacs = ... # loop over the constraints and take the jacobian

f_fused = ... # return objective and objective_grad together
f_hess = ... # return only the hessian

constraint_fns = ... # Make a list of compiled functions. They should all take budget!
constraint_jac_fns = ... # Same thing -- everyone takes budget!

constraints = []
budget_value = 1.0

if not constraint_fns is Ellipsis:
    for typ, fun, jac in zip(constraint_types, constraint_fns, constraint_jac_fns):
        constraints.append({'type':typ, 'fun':fun, 'jac':jac, 'args': (budget_value, )})

In [23]:
def do_minimization(x0, alphas, lambdas):
    return minimize(
        fun = ...,
        jac = ..., 
        hess = ..., 
        constraints = ...,
        x0 = ...,
        args= ...,
        method='trust-constr',
        options={'maxiter':100_000})

In [26]:
@test
def test_constrainted_optimization(minimize_func, constraints):
    expected_result = np.array([ 2.415e-01,  3.722e-06,  1.194e-01,  2.126e-01,  
                                2.421e-01, 8.706e-07,  1.843e-01,  6.885e-07])
    alphas = np.array([0.5, 0.3, 0.3, 0.5, 0.5, 0.4, 0.4, 0.3])
    lambdas = np.array([0.3, 0.6, 0.3, 0.1, 0.2, 0.9, 0.3, 0.7])
    x0 = np.ones_like(alphas) / alphas.shape[0]

    res = minimize_func(x0, alphas, lambdas)
    
    assert res.success
    np.testing.assert_allclose(res.fun, -1.07, atol=0.01, rtol=0.01)    
    np.testing.assert_allclose(res.x, expected_result, atol=0.01, rtol=0.01)    
    
    for constraint in constraints:
        x_constraint = constraints['fun'](res.x, budget_value)
        if constraint['type'] == 'eq':
            assert np.testing.allclose(x_constraint, 0.0)
        else:
            assert all(x_constraint >= 0)
    

# test_constrainted_optimization(do_minimization, constraints)

In [27]:
# If you passed the last test, this cell will run happily
alphas = np.array([0.5, 0.3, 0.3, 0.5, 0.5, 0.4, 0.4, 0.3])
lambdas = np.array([0.3, 0.6, 0.3, 0.1, 0.2, 0.9, 0.3, 0.7])
x0 = np.ones_like(alphas) / alphas.shape[0]

EXCERISE_8_PASSED = False

try: 
    res = do_minimization(x0, alphas, lambdas)
    EXCERISE_8_PASSED = True
    
except Exception:
    pass

## Exercise 8.5: Plot the results

How did they match up to your intuitions in 7.5? How did your high score compare to the optimizer?

In [28]:
if EXCERISE_8_PASSED:
    %matplotlib inline
    fig, ax = plt.subplots(1, 2, figsize=(14, 4), dpi=77)

    x_grid = np.linspace(0, 1, 100)
    allocations = res.x

    ax[0].plot(x_grid, f_mm_vec_2(x_grid, alphas, lambdas), label=[f'x{i+1}' for i in range(8)])
    ax[0].set_title('Utility from Channel Allocations')
    ax[0].legend()

    initial_utility = np.diagonal(f_mm_vec_2(allocations, alphas, lambdas))
    scatter = ax[0].scatter(allocations, initial_utility)

    spend_bar, utility_bar = ax[1].bar(['Total Spend', 'Total Utility'], 
              [allocations.sum(),                          
               np.trace(f_mm_vec_2(allocations, alphas, lambdas))])

    ax[1].axhline(1.0, ls='--', c='k', lw=2, label='Total Budget')
    ax[1].set(ylim=(0, 1.5), title=f'High Score: {initial_utility.sum()}')
    ax[1].legend()
    plt.show()

## Final Round: Pytensor for organization

One detail I've elided so far is that there might be more structure to the curves, and we might want to have more constraints across that structure. 

Let's imagine that we have five advertising channels: TV, Facebook, TikTok, whatever. 
And we have two products: A and B.

Not all products are in all channels. Our eight parameters correspond to:

1. Product A, Channel 0
2. Product A, Channel 1
3. Product A, Channel 2
4. Product A, Channel 3
5. Product A, Channel 4
6. Product B, Channel 0
7. Product B, Channel 1
8. Product B, Channel 2

To organize all this, we want to put our trusty `x_vector` into a matrix. We can do it with https://numpy.org/doc/stable/user/basics.indexing.html#advanced-indexing. This works the same in pytensor as numpy, but we will have to use the `.set` method to help.

What follows is a simple example, setting the main diagonal of a matrix to a vector.

In [29]:
X_example = pt.zeros((5, 5))
values = pt.arange(5)
X_example = X_example[pt.arange(5), pt.arange(5)].set(values)
X_example.eval()

array([[0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 2., 0., 0.],
       [0., 0., 0., 3., 0.],
       [0., 0., 0., 0., 4.]])

We can use exactly the same logic to allocate `x_vector` to a matrix with shape `(n_products, n_channels)`. Then, when we want to impose spending limits on total across products or channels, we just use the corresponding row or column sum.

In [30]:
n_products = 2
n_channels = 5
X_matrix = ... # Initialize this with zeros
row_idxs = ... # Numpy array of 0 and 1, corresponding to the product idx of each x. Length should be the same as
               # that of x_vector
col_idxs = ... # Numpy array of values between 0 and 4, corresponding to the channel idx of each x. 
               # Length should be the same as that of x_vector

X_matrix = ... # Use indexing and .set(x_vector) !

In [31]:
n_products = 2
n_channels = 5
X_matrix = pt.zeros((n_products, n_channels))
row_idx = np.array([0, 0, 0, 0, 0, 1, 1, 1])
col_idx = np.array([0, 1, 2, 3, 4, 0, 1, 2])

X_matrix = X_matrix[row_idx, col_idxs].set(x_vector)

Using `X_matrix`, create the following constraints:

- Allocation across a single channel should be between 10% and 70% of the total budget
- Maximum total allocation to channel 3 is 0.1
- Maximum total allocation to channel 2 is 0.5
- Minimum total allocation to product 0 is 0.5
- Minimum total allocation to product 1 is 0.4
- The sum of all allocations equals 1.0
- All allocations are all at least 5% of the total budget

In total, this will make 8 constraints (because the first one is two constraints). Remember that inequality constraints are always of the form $f(x) \geq 0$ !

In [32]:
constraint_types = ['eq'] + ['ineq'] * 7


constraint_graphs = ...
constraint_jacs = ...

constraint_fns = ... # Hint: the input is still x_vector, even though we use X_matrix in the constraints!
constraint_jac_fns = ... # Remember that everyone takes budget!

constraints = []
budget_value = 1.0

if not constraint_fns is Ellipsis:
    for typ, fun, jac in zip(constraint_types, constraint_fns, constraint_jac_fns):
        constraints.append({'type':typ, 'fun':fun, 'jac':jac, 'args': (budget_value, )})

In [ ]:
@test
def test_constrainted_optimization_again(minimize_func, constraints):
    expected_result = np.array([1.576e-01, 5.000e-02, 5.450e-02, 1.642e-01,
                                1.737e-01, 8.221e-02, 2.671e-01, 5.071e-02])
    alphas = np.array([0.5, 0.3, 0.3, 0.5, 0.5, 0.4, 0.4, 0.3])
    lambdas = np.array([0.3, 0.6, 0.3, 0.1, 0.2, 0.9, 0.3, 0.7])
    x0 = np.ones_like(alphas) / alphas.shape[0]

    res = minimize_func(x0, alphas, lambdas)
    
    assert res.success
    np.testing.assert_allclose(res.fun, -1.02, atol=0.01, rtol=0.01)    
    np.testing.assert_allclose(res.x, expected_result, atol=0.01, rtol=0.01)    
    
    for constraint in constraints:
        x_constraint = constraint['fun'](res.x, budget_value)
        if constraint['type'] == 'eq':
            np.testing.assert_allclose(x_constraint, 0.0, atol=0.01, rtol=0.01)
        else:
            assert all(x_constraint >= 0)

In [ ]:
def do_minimization_again(x0, alphas, lambdas):
    return minimize(
        fun = ...,
        jac = ..., 
        hess = ..., 
        constraints = ...,
        x0 = ...,
        args= ...,
        method='trust-constr',
        options={'maxiter':100_000})

test_constrainted_optimization_again(do_minimization_again, constraints)

In [ ]:
# If you passed the last test, this cell will run happily
alphas = np.array([0.5, 0.3, 0.3, 0.5, 0.5, 0.4, 0.4, 0.3])
lambdas = np.array([0.3, 0.6, 0.3, 0.1, 0.2, 0.9, 0.3, 0.7])
x0 = np.ones_like(alphas) / alphas.shape[0]

EXCERISE_9_PASSED = False

try: 
    new_res = do_minimization_again(x0, alphas, lambdas)
    EXCERISE_9_PASSED = True
    
except Exception:
    pass

In [ ]:
if EXCERISE_9_PASSED:
    %matplotlib inline
    fig, ax = plt.subplots(1, 2, figsize=(14, 4), dpi=77)

    x_grid = np.linspace(0, 1, 100)

    allocations = res.x
    new_allocations = new_res.x

    ax[0].plot(x_grid, f_mm_vec_2(x_grid, alphas, lambdas), label=[f'x{i+1}' for i in range(8)])
    ax[0].set_title('Utility from Channel Allocations')
    ax[0].legend()

    utility = np.diagonal(f_mm_vec_2(allocations, alphas, lambdas))
    new_utility = np.diagonal(f_mm_vec_2(new_allocations, alphas, lambdas))
    ax[0].scatter(allocations, initial_utility, color='tab:blue')
    ax[0].scatter(new_allocations, new_utility, color='tab:red')


    ax[1].bar(['Total Spend', 'Total Utility'], 
              [allocations.sum(), np.trace(f_mm_vec_2(allocations, alphas, lambdas))])

    ax[1].axhline(1.0, ls='--', c='k', lw=2, label='Total Budget')
    ax[1].set(ylim=(0, 1.5), title=f'New Utility: {new_utility.sum()}')
    ax[1].legend()
    plt.show()